## Contexto do problema

Instituições financeiras liram diariamente com milhares de operações internas e transações que, quando mal monitoradas, podem gerar **risco operacional**.

Risco operacional envolve falhas em processos, sistemas, pessoas ou eventos externos, podendo resultar em:
- perdas financeiras
- falhas de compliance 
- impactos reputacionais

## Cenário simulado 

Neste projeto, será simulada uma base de dados contendo registros operacionais de uma instituição financeira fictícia 

Cada registro representa uma ocorrência operacional associada a:
- área responsável
- tipo de evento 
- data
- valor financeiro impactado 
- canal de origem 

O objetivo é identificar padrões que possam indicar **aumento de risco operacional**.

## Estrutura dos Dados

Cada linha do dataset representa uma ocorrência de risco operacional registrada pela instituição

A seguir estão descritas as principais variáveis consideradas na análise.

## Variáveis do Dataset 

- **id_ocorrencia**: identificador único da ocorrência
- **data_ocorrencia**: data em que o evento foi registrado 
- **area_responsavel**: área interna responsável pelo processo
- **tipo_evento**: classificação do evento operacional 
- **canal_origem**: canal onde o evento ocorreu (ex: digital, agência, backoffice)
- **valor_impacto**: valor financeiro estimado do impacto
- **nivel_risco**: classificação do risco (baixo, médio, alto)

In [ ]:
import random
import pandas as pd
import numpy as np
from datetime import datetime, timedelta 

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

import psycopg2
from psycopg2 import sql, Error
from config import DB_CONFIG

In [ ]:
def sortear_nivel_risco():
    return random.choices(
        ["baixo", "medio", "alto", "critico"],
        weights=[0.6, 0.25, 0.1, 0.05]
    )[0]

def gerar_impacto_por_nivel(nivel):
    if nivel == "baixo":
        return random.randint(1_000, 50_000)
    elif nivel == "medio":
        return random.randint(50_001, 250_000)
    elif nivel == "alto":
        return random.randint(250_001, 1_000_000)
    else:
        return random.randint(1_000_001, 5_000_000)
    
def gerar_evento():
    nivel = sortear_nivel_risco()
    impacto = gerar_impacto_por_nivel(nivel)

    dias_atras = random.randint(0, 30)
    hora_aleatoria = random.randint(0, 23)
    minuto_aleatorio = random.randint(0, 59)
    
    data_evento = datetime.now() - timedelta(
        days=dias_atras,
        hours=hora_aleatoria,
        minutes=minuto_aleatorio
    )
    
    if nivel == "baixo":
        horas_resolucao = random.randint(1, 24)
    elif nivel == "medio":
        horas_resolucao = random.randint(4, 48)
    elif nivel == "alto":
        horas_resolucao = random.randint(12, 72)
    else:
        horas_resolucao = random.randint(24, 168)
    
    data_resolucao = data_evento + timedelta(hours=horas_resolucao)

    evento = {
        "data_evento": data_evento,
        "data_resolucao": data_resolucao,
        "tempo_resolucao_horas": horas_resolucao,
        "impacto_financeiro": impacto,
        "impacto_cliente": random.randint(0, 1),
        "tempo_indisponibilidade": round(random.uniform(0.1, 8), 2),
        "frequencia_evento": random.randint(1, 10),
        "criticidade_sistema": random.randint(1, 5),
        "falha_processo": random.randint(0, 1),
        "fraude_interna": random.randint(0, 1),
        "recorrencia": random.randint(1, 5),
        "nivel_risco": nivel
    }
    
    return evento

In [ ]:
eventos = [gerar_evento() for _ in range(5_000)]
df = pd.DataFrame(eventos)

In [ ]:
contagem = df["nivel_risco"].value_counts()
percentual = df["nivel_risco"].value_counts(normalize=True) * 100

resultado = pd.DataFrame({
    "quantidade": contagem,
    "percentual (%)": percentual.round(2)
})

print("\n📊 DISTRIBUIÇÃO DOS EVENTOS")

resultado

In [ ]:
# Preparação dos dados para ML
X = df.drop(columns=["nivel_risco", "data_evento", "data_resolucao"])
y = df["nivel_risco"]

print("\n📊 Features do modelo: \n")
print(X.dtypes)
print(f"\n✅ Shape: {X.shape}")
print(f"✅ Target: {y.name} com {y.nunique()} classes")

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

print("\n📊 Divisão dos dados: \n")
print(f"Treino: {x_train.shape[0]} eventos")
print(f"Teste: {x_test.shape[0]} eventos")
print(f"Features: {x_train.shape[1]}")

In [ ]:
# Treinamento do modelo
print("🤖 Treinando Random Forest...")

modelo = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    class_weight="balanced"
)

modelo.fit(x_train, y_train)

print("✅ Modelo treinado com sucesso!")

# Mostra os parâmetros do modelo
modelo

In [ ]:
# Avaliação do modelo
y_pred = modelo.predict(x_test)

print("\n📊 RELATÓRIO DE CLASSIFICAÇÃO")
print("=" * 60)
print(classification_report(y_test, y_pred))

In [ ]:
df.shape

In [ ]:
# criar tabela no postgree

def criar_tabela_eventos():
    """
    Cria tabela para armazenar eventos de risco operacional
    """
    try:
        # conecta
        conexao = psycopg2.connect(**DB_CONFIG)
        print('✅ PostgreSQL conectado com sucesso!')
        print(f'✅ Versão: {conexao.server_version}')

        cursor = conexao.cursor()

        # SQL para criar tabela
        query = """
        CREATE TABLE IF NOT EXISTS eventos_risco (
            id SERIAL PRIMARY KEY,
            evento_id VARCHAR(100) UNIQUE NOT NULL,
            data_evento TIMESTAMP NOT NULL,
            data_resolucao TIMESTAMP,
            tempo_resolucao_horas FLOAT,
            impacto_financeiro DECIMAL(15, 2) NOT NULL,
            impacto_cliente INTEGER,
            tempo_indisponibilidade FLOAT,
            frequencia_evento INTEGER,
            criticidade_sistema INTEGER,
            falha_processo INTEGER,
            fraude_interna INTEGER,
            recorrencia INTEGER,
            nivel_risco VARCHAR(20) NOT NULL,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
        
        CREATE INDEX IF NOT EXISTS idx_nivel_risco ON eventos_risco(nivel_risco);
        CREATE INDEX IF NOT EXISTS idx_data_evento ON eventos_risco(data_evento);
        CREATE INDEX IF NOT EXISTS idx_impacto ON eventos_risco(impacto_financeiro);
        """

        # executa
        cursor.execute(query)
        conexao.commit()

        print('✅ Tabela eventos_risco criada com sucesso!')
        print('✅ índices criados para otimizar consultas!')

        cursor.close()
        conexao.close()

        return True
    except Exception as e:
        print(f'❌ Erro ao criar tabela: {e}')
        return False
    
# cria a tabela
criar_tabela_eventos()

✅ PostgreSQL conectado com sucesso!
✅ Versão: 180001
✅ Tabela eventos_risco criada com sucesso!
✅ índices criados para otimizar consultas!


True